In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.dt_agent import DTAgent
from sklearn.model_selection import train_test_split

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
np.random.seed(SEED)

#### The data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_8.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.317680,-1.000000,2.088832,-1.000000,-1.000000,90.986272,113.064122,3.731666,1,1.276701,81.145964,78.146183,29.513433,26.029977,105.808765,33.953040,-1.000000,Aplastic anemia
1,10.109186,-1.000000,-1.000000,-1.000000,-1.000000,104.941225,107.239455,2.889957,0,1.191488,86.444945,103.728730,32.453257,20.009164,90.105559,30.327558,-1.000000,Unspecified anemia
2,6.289434,-1.000000,4.156000,-1.000000,-1.000000,86.013564,-1.000000,2.193642,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,18.868301,-1.000000,Hemolytic anemia
3,7.077683,97.195746,0.261727,0.142422,263.478460,94.058079,102.335651,2.257440,1,-1.000000,114.702341,64.345736,20.102258,3.865156,-1.000000,21.233048,38.840234,No anemia
4,9.717874,101.889169,-1.000000,3.937127,475.993105,78.258930,-1.000000,3.725277,0,1.367858,-1.000000,-1.000000,38.308559,-1.000000,-1.000000,29.153622,-1.000000,No anemia


In [4]:
utils.get_dt_performance(df)

(0.6207142857142857,
 0.6122228660331646,
 0.7775530397555184,
 datetime.timedelta(microseconds=5024))

In [5]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train_df), np.array(y_train_df)
X_test, y_test = np.array(X_test_df), np.array(y_test_df)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [6]:
y_test[:5]

array([4, 4, 5, 7, 5], dtype=int64)

#### The DT agent

In [7]:
%%time
dt_agent = DTAgent(X_test, y_test)
test_df = dt_agent.test()
test_df.head()

Testing done.....
Wall time: 3min 28s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,5.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, ferritin, tibc, Iron deficie...",4.0,4.0
1,5.0,1.0,1.0,1.0,0.0,"[hemoglobin, mcv, ferritin, tibc, Iron deficie...",4.0,4.0
2,5.0,2.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, ret_count, Hemolytic...",5.0,5.0
3,4.0,3.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Inconclusive diag...",7.0,7.0
4,4.0,4.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0


In [8]:
# %%time
# dt_agent.test_sample(2)

In [9]:
# from datetime import datetime
# start = datetime.now()
# dt_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [10]:
# X_test_df.head()

In [11]:
len(test_df), len(X_test)

(14000, 14000)

In [12]:
test_df.to_csv('../../test_dfs/many_features/0.1/correlated/dt_test_df_corr_noisy_8.csv', index=False)

In [13]:
# test_df = pd.read_csv('../../test_dfs/many_features/dt_test_df_0.1.csv')
# test_df.head()

In [14]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

62.76428571428572

In [15]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.013, 0.2552857142857143)

In [16]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.6276428571428572, 0.6199216163840183, 0.7994301659335363)

In [ ]:
# utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])